In [ ]:
import tensorflow as tf
import numpy as np
import keras 
import pandas as pd

In [ ]:
data=pd.read_csv('/content/drive/My Drive/dataset/ttran.csv')

In [ ]:
data.drop('id',axis=1,inplace=True)

In [ ]:
llabel=[]
llabel1=[]
llabel2=[]
llabel3=[]
llabel4=[]
llabel5=[]

In [ ]:
llabel=list(data['toxic'])
llabel1=list(data['severe_toxic'])
llabel2=list(data['obscene'])
llabel3=list(data['threat'])
llabel4=list(data['insult'])
llabel5=list(data['identity_hate'])

In [ ]:
labell=[]

In [ ]:
for a,b,c,d,e,f in zip(llabel,llabel1,llabel2,llabel3,llabel4,llabel5):
  labell.append([a,b,c,d,e,f])

In [ ]:
import re
def preprocess_sentence(w):
  w = w.lower()
  w = re.sub(r"([?.!,¿])", r" \1 ", w)
  w = re.sub(r'[" "]+', " ", w)
  w = re.sub(r"[^a-zA-Z?.!,¿]+", " ", w)
  w = w.strip()
  w = '<start> ' + w + ' <end>'
  return w

In [ ]:
texts=[]
a=0

In [ ]:
for line in data['comment_text']:
  a=preprocess_sentence(line)
  texts.append(a)

In [ ]:
from keras.preprocessing import text, sequence

In [ ]:
max_features = 30000
maxlen = 100
embed_size = 300

tokenizer = text.Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(texts)

X_train = tokenizer.texts_to_sequences(texts)
x_train1 = sequence.pad_sequences(X_train, maxlen=maxlen,padding="post")



In [ ]:
b=tf.keras.layers.LSTM(256)
dd=tf.keras.layers.Embedding(30000,256)
d=tf.keras.layers.Dense(64)
m=tf.keras.layers.Flatten()
ccc=tf.keras.layers.GlobalMaxPool1D()

In [ ]:
from keras import backend as K
from keras.engine.topology import Layer
from keras.models import Model
from keras.layers import *

In [ ]:
class Position_Embedding(Layer):

  def __init__(self, size=None, mode='sum', **kwargs):
    self.size = size 
    self.mode = mode
    super(Position_Embedding, self).__init__(**kwargs)

  def call(self, x):
    if (self.size == None) or (self.mode == 'sum'):
      self.size = int(x.shape[-1])
    batch_size, seq_len = K.shape(x)[0], K.shape(x)[1]
    position_j = 1. / K.pow(10000., 2 * K.arange(self.size / 2, dtype='float32') / self.size)
    position_j = K.expand_dims(position_j, 0)
    position_i = K.cumsum(K.ones_like(x[:, :, 0]),1) - 1 
    position_i = K.expand_dims(position_i, 2)
    position_ij = K.dot(position_i, position_j)
    position_ij = K.concatenate([K.cos(position_ij), K.sin(position_ij)], 2)
    if self.mode == 'sum':
      return position_ij + x
    elif self.mode == 'concat':
      return K.concatenate([position_ij, x], 2)

  def compute_output_shape(self, input_shape):
    if self.mode == 'sum':
      return input_shape
    elif self.mode == 'concat':
      return (input_shape[0], input_shape[1], input_shape[2] + self.size)

In [ ]:
def scaled_dot_product_attention(q, k, v, mask=None):
  matmul_qk = tf.matmul(q, k, transpose_b=True)  
  dk = tf.cast(tf.shape(k)[-1], tf.float32)
  scaled_attention_logits = matmul_qk / tf.math.sqrt(dk)
  if mask is not None:
    scaled_attention_logits += (mask * -1e9)  
  attention_weights = tf.nn.softmax(scaled_attention_logits, axis=-1) 
  output = tf.matmul(attention_weights, v)  
  return output, attention_weights

In [ ]:
class MultiHeadAttention(tf.keras.layers.Layer):
  def __init__(self, d_model, num_heads):
    super(MultiHeadAttention, self).__init__()
    self.num_heads = num_heads
    self.d_model = d_model
    assert d_model % self.num_heads == 0
    self.depth = d_model // self.num_heads
    self.wq = tf.keras.layers.Dense(d_model)
    self.wk = tf.keras.layers.Dense(d_model)
    self.wv = tf.keras.layers.Dense(d_model)
    self.dense = tf.keras.layers.Dense(d_model)
  def split_heads(self, x, batch_size):
    x = tf.reshape(x, (batch_size, -1, self.num_heads, self.depth))
    return tf.transpose(x, perm=[0, 2, 1, 3])
  def call(self, v, k, q, mask):
    batch_size = tf.shape(q)[0]
    q = self.wq(q)  
    k = self.wk(k)    
    v = self.wv(v)    
    q = self.split_heads(q, batch_size)  
    k = self.split_heads(k, batch_size)  
    v = self.split_heads(v, batch_size) 
    scaled_attention, attention_weights = scaled_dot_product_attention(q, k, v, mask)
    scaled_attention = tf.transpose(scaled_attention, perm=[0, 2, 1, 3]) 
    concat_attention = tf.reshape(scaled_attention, (batch_size, -1, self.d_model))  
    output = self.dense(concat_attention)  
    return output, attention_weights

In [ ]:
S_inputs = Input(shape=([None]), dtype='int32')
embeddings = dd(S_inputs)

c=b(embeddings)
O_seq,a = MultiHeadAttention(512, 8)(c,c,c,None)
b_seq=d(O_seq)
e = Dropout(0.2)(b_seq)
e=ccc(e)
outputs = Dense(6, activation='softmax')(e)
model = Model(inputs=S_inputs, outputs=outputs)

In [ ]:
model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, None, 256)    7680000     input_1[0][0]                    
__________________________________________________________________________________________________
lstm (LSTM)                     (None, 256)          525312      embedding[0][0]                  
__________________________________________________________________________________________________
multi_head_attention (MultiHead ((None, None, 512),  657408      lstm[0][0]                       
                                                                 lstm[0][0]            

In [ ]:
from sklearn.model_selection import train_test_split
X_tra, X_val, y_tra, y_val = train_test_split(
    x_train1, labell, test_size=0.3, random_state=233)

In [ ]:
X_tain1=tf.data.Dataset.from_tensor_slices((X_tra,y_tra))

In [ ]:
X_val1=tf.data.Dataset.from_tensor_slices((X_val,y_val))

In [ ]:
XX_t=X_tain1.batch(32).shuffle(10000)
XX_v=X_val1.batch(32).shuffle(10000)

In [ ]:
loss_fn=tf.keras.losses.CategoricalCrossentropy(from_logits=True)

In [ ]:
model.compile(loss=loss_fn, optimizer='adam', metrics=['accuracy'])

In [ ]:
hist = model.fit(XX_t,batch_size=32,epochs=50,validation_data=(XX_v))

Epoch 1/50
3491/3491 [==============================] - 298s 85ms/step - loss: 0.3655 - accuracy: 0.9516 - val_loss: 0.3612 - val_accuracy: 0.9938
Epoch 2/50
3491/3491 [==============================] - 299s 86ms/step - loss: 0.3645 - accuracy: 0.8946 - val_loss: 0.3575 - val_accuracy: 0.5382
Epoch 3/50
3491/3491 [==============================] - 303s 87ms/step - loss: 0.3620 - accuracy: 0.7956 - val_loss: 0.3553 - val_accuracy: 0.8988
Epoch 4/50
 360/3491 [==>...........................] - ETA: 4:22 - loss: 0.3576 - accuracy: 0.6422

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py", line 2882, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-26-020369656f56>", line 1, in <module>
    hist = model.fit(XX_t,batch_size=32,epochs=50,validation_data=(XX_v))
  File "/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/training.py", line 108, in _method_wrapper
    return method(self, *args, **kwargs)
  File "/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/training.py", line 1098, in fit
    tmp_logs = train_function(iterator)
  File "/usr/local/lib/python3.6/dist-packages/tensorflow/python/eager/def_function.py", line 780, in __call__
    result = self._call(*args, **kwds)
  File "/usr/local/lib/python3.6/dist-packages/tensorflow/python/eager/def_function.py", line 807, in _call
    return self._stateless_fn(*args, **kwds)  # pylint: disable=not-callable
  File "/usr/local/lib/

KeyboardInterrupt: ignored